In [1]:
import os
import errno
import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks

import deepcell
from deepcell import model_zoo
from deepcell import losses
from deepcell.utils.data_utils import get_data
from deepcell.utils.train_utils import rate_scheduler
from deepcell.utils.tracking_utils import load_trks
from deepcell.utils.tracking_utils import save_trks
from deepcell.utils.retinanet_anchor_utils import get_anchor_parameters
from deepcell.callbacks import RedirectModel, Evaluate
from deepcell.image_generators import RetinaMovieDataGenerator, RetinaNetGenerator
# from deepcell.model_zoo import shapemask_box

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Helper Functions
From `shape_mask` branch

In [2]:
def train_model(model,
                model_dir=None,
                model_name=None,
                train_dict=None,
                test_dict=None,
                batch_size=1,
                num_classes=1,
                fpb=1,
                backbone_levels=None,
                pyramid_levels=None,
                anchor_params=None,
                n_epoch=16,
                optimizer=Adam(lr=1e-5, clipnorm=0.001),
                lr_sched = rate_scheduler(lr=1e-5, decay=0.99)
                ):
    
    if fpb == 1:
        datagen = RetinaNetGenerator(
            rotation_range=180,
            zoom_range=(0.8, 1.2),
            horizontal_flip=True,
            vertical_flip=True)

        datagen_val = RetinaNetGenerator()

        train_data = datagen.flow(
            train_dict,
            batch_size=batch_size,
            include_masks=True,
            include_final_detection_layer=True,
            pyramid_levels=pyramid_levels,
            anchor_params=anchor_params)

        val_data = datagen_val.flow(
            test_dict,
            batch_size=batch_size,
            include_masks=True,
            include_final_detection_layer=True,
            pyramid_levels=pyramid_levels,
            anchor_params=anchor_params)
    
    else:
        datagen = RetinaMovieDataGenerator(
            rotation_range=180,
            zoom_range=(0.8, 1.2),
            horizontal_flip=True,
            vertical_flip=True)

        datagen_val = RetinaMovieDataGenerator()

        train_data = datagen.flow(
            train_dict,
            batch_size=batch_size,
            include_masks=True,
            include_final_detection_layer=True,
            frames_per_batch=fpb,
            pyramid_levels=pyramid_levels,
            anchor_params=anchor_params)

        val_data = datagen_val.flow(
            test_dict,
            batch_size=batch_size,
            include_masks=True,
            include_final_detection_layer=True,
            frames_per_batch=fpb,
            pyramid_levels=pyramid_levels,
            anchor_params=anchor_params)

    retinanet_losses = losses.RetinaNetLosses(
        sigma=3.0,
        alpha=0.25,
        gamma=2.0,
        iou_threshold=0.5,
        mask_size=(28,28))

    loss = {
        'regression': retinanet_losses.regress_loss,
        'classification': retinanet_losses.classification_loss,
        'masks': retinanet_losses.mask_loss,
        'final_detection': retinanet_losses.final_detection_loss
        }

    model.compile(loss=loss, optimizer=optimizer)

    iou_threshold = 0.5
    score_threshold = 0.01
    max_detections = 100

    model.fit_generator(
        train_data,
        steps_per_epoch=X_train.shape[0] * X_train.shape[1]// batch_size,
        epochs=n_epoch,
        validation_data=val_data,
        validation_steps=X_test.shape[0] * X_test.shape[1]// batch_size,
        callbacks=[
            callbacks.LearningRateScheduler(lr_sched),
            callbacks.ModelCheckpoint(
                os.path.join(model_dir, model_name + '.h5'),
                monitor='val_loss',
                verbose=1,
                save_best_only=True,
                save_weights_only=False),
            RedirectModel(
                Evaluate(val_data,
                         iou_threshold=iou_threshold,
                         score_threshold=score_threshold,
                         max_detections=max_detections,
                         frames_per_batch=fpb,
                         weighted_average=True),
                prediction_model)
        ])

    return None

# Train models

In [ ]:
# download_datasets()

DATA_DIR = '/data/training_data/tracking_benchmark_data'

backbones = ['featurenet'] #, 'mobilenetv2', 'resnet50']
fpbs = [5, 3, 1]
all_data = '3T3_HeLa_HEK_RAW_cropped.npz'
datasets = [all_data]
temporal_modes = ['conv', 'gru', 'lstm', None]
shape_mask = False

n_epoch = 4
seed = 808


for dataset in datasets:
    num_classes=1
    test_size = 0.1 # % of data saved as test
    test_seed = 10

    filename = os.path.join(DATA_DIR, dataset)
    train_dict, test_dict = get_data(filename, seed=seed, test_size=test_size)
    print(' -\nX.shape: {}\ny.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))
    X_train, y_train = train_dict['X'], train_dict['y']
    X_test, y_test = test_dict['X'], test_dict['y']
    y_train_reshaped = y_train.reshape((-1,  X_train.shape[2], X_train.shape[3], X_train.shape[4]))
    print("y_train_reshaped shape:", y_train_reshaped.shape)
    optimal_params = get_anchor_parameters(y_train.reshape((-1,  X_train.shape[2], X_train.shape[3], X_train.shape[4])))
    backbone_levels, pyramid_levels, anchor_params = optimal_params
    norm_method='whole_image'
    print("optimal_params: ", optimal_params)

    for backbone in backbones:
        if backbone == 'featurenet':
            use_imagenet=False
        else:
            use_imagenet=True

        for fpb in fpbs:
            if fpb == 1:
                train_dict = {'X':X_train.reshape((-1,X_train.shape[2], X_train.shape[3], X_train.shape[4])), 
                              'y': y_train.reshape((-1,y_train.shape[2], y_train.shape[3], y_train.shape[4]))}
                test_dict = {'X':X_test.reshape((-1, X_test.shape[2], X_test.shape[3], X_test.shape[4])), 
                            'y': y_test.reshape((-1, y_test.shape[2], y_test.shape[3], y_test.shape[4]))}
            else:
                train_dict = {'X':X_train, 'y':y_train}
                test_dict = {'X':X_test, 'y':y_test}
            print(' -\nX.shape: {}\ny.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))


            for temporal_mode in temporal_modes:
                model = model_zoo.RetinaMask(backbone=backbone,
                                        use_imagenet=use_imagenet,
                                        panoptic=False,
                                        frames_per_batch=fpb,
                                        temporal_mode=temporal_mode,
                                        num_classes=num_classes,
                                        input_shape=X_train.shape[2:],
                                        anchor_params=anchor_params,
                                        class_specific_filter=False,
                                        backbone_levels=backbone_levels,
                                        pyramid_levels=pyramid_levels,
                                        norm_method=norm_method)
                prediction_model = model

                model_dir = '/data/models'
                model_name = backbone + '_' + 'fpb' + str(fpb) + '_' + str(temporal_mode) + '_' + dataset

                # Train model
                print("Training model: ", model_name)
                trained_model = train_model(model,
                            model_dir=model_dir,
                            model_name=model_name,
                            train_dict=train_dict,
                            test_dict=test_dict,
                            fpb=fpb,
                            backbone_levels=backbone_levels,
                            pyramid_levels=pyramid_levels,
                            anchor_params=anchor_params,
                            n_epoch=n_epoch,
                          )


 -
X.shape: (722, 30, 135, 160, 1)
y.shape: (722, 30, 135, 160, 1)
y_train_reshaped shape: (21660, 135, 160, 1)


W0110 22:30:00.568224 140563580065600 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


optimal_params:  (['C1', 'C2', 'C3'], ['P1', 'P2', 'P3'], <deepcell.utils.retinanet_anchor_utils.AnchorParameters object at 0x7fd778187860>)
 -
X.shape: (722, 30, 135, 160, 1)
y.shape: (722, 30, 135, 160, 1)


W0110 22:30:04.274227 140563580065600 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0110 22:30:11.921495 140563580065600 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:255: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0110 22:30:13.010686 140563580065600 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:255: calling crop_and_resize_v1 (from tensorflow.python.ops.image_ops_impl) with box_i

Training model:  featurenet_fpb5_conv_3T3_HeLa_HEK_RAW_cropped.npz


W0110 22:30:22.306921 140563580065600 retinanet.py:619] Removing 77 of 722 images with fewer than 3 objects.
W0110 22:30:25.065605 140563580065600 retinanet.py:619] Removing 7 of 81 images with fewer than 3 objects.
W0110 22:30:25.283152 140563580065600 training_utils.py:1101] Output filtered_detections missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to filtered_detections.
W0110 22:30:25.284173 140563580065600 training_utils.py:1101] Output filtered_detections_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to filtered_detections_1.
W0110 22:30:25.285067 140563580065600 training_utils.py:1101] Output filtered_detections_2 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to filtered_detections_2.
W0110 22:30:25.285877 1405635

Epoch 1/4
 9836/21660 [============>.................] - ETA: 3:00:58 - loss: 2.0360 - regression_loss: 1.1410 - classification_loss: 0.2829 - masks_loss: 0.3185 - final_detection_loss: 0.2936

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



21659/21660 [============================>.] - ETA: 0s - loss: 1.7052 - regression_loss: 0.9588 - classification_loss: 0.2251 - masks_loss: 0.2964 - final_detection_loss: 0.2250
Epoch 00001: val_loss improved from inf to 1.27951, saving model to /data/models/featurenet_fpb5_conv_3T3_HeLa_HEK_RAW_cropped.npz.h5
718 instances of class 0 with average precision: 0.9117
mAP: 0.9117
21660/21660 [==============================] - 20875s 964ms/step - loss: 1.7053 - regression_loss: 0.9588 - classification_loss: 0.2251 - masks_loss: 0.2964 - final_detection_loss: 0.2250 - val_loss: 1.2795 - val_regression_loss: 0.7595 - val_classification_loss: 0.1161 - val_masks_loss: 0.2507 - val_final_detection_loss: 0.1532
Epoch 2/4
 4289/21660 [====>.........................] - ETA: 4:25:16 - loss: 1.3164 - regression_loss: 0.7413 - classification_loss: 0.1636 - masks_loss: 0.2677 - final_detection_loss: 0.1437

# Benchmark

In [6]:
# Define data to load (raw images from trk test files)
RAW_BASE_DIR = '/data/training_data/tracking_benchmark_data/test'

raw_trks_3T3  = os.path.join(RAW_BASE_DIR, '3T3_NIH_test_BData.trks')
raw_trks_HEK  = os.path.join(RAW_BASE_DIR, 'HEK293_generic_test_BData.trks')
raw_trks_HeLa = os.path.join(RAW_BASE_DIR, 'HeLa_S3_test_BData.trks')
raw_trks_RAW  = os.path.join(RAW_BASE_DIR, 'RAW264_generic_test_BData.trks')

# raw_trks_files = [raw_trks_3T3, raw_trks_HEK, raw_trks_HeLa]
raw_trks_files = [raw_trks_3T3, raw_trks_HEK, raw_trks_HeLa, raw_trks_RAW]

model_dir = '/data/models'
model_name = backbone + '_' + 'fpb' + str(fpb) + '_' + str(temporal_mode) + '_' + dataset

In [ ]:
# Make predictions on test data
from deepcell.utils.tracking_utils import load_trks
from skimage.morphology import remove_small_objects
import pandas as pd
from deepcell import metrics
from skimage.measure import label
from skimage import morphology
from skimage.morphology import watershed
from skimage.feature import peak_local_max

backbones = ['featurenet'] #, 'mobilenetv2', 'resnet50']
fpbs = [5, 3, 1]
all_data = '3T3_HeLa_HEK_RAW_cropped.npz'
datasets = [all_data]
temporal_modes = ['conv', 'gru', 'lstm', None]
shape_mask = False

n_epoch = 4
seed = 808

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(raw_trks_files):
    # Load the trk file       
    trks = load_trks(dataset)
    lineages, raw, tracked = trks['lineages'], trks['X'], trks['y']
    optimal_params = get_anchor_parameters(trks['y'].reshape((-1,  trks['X'].shape[2], trks['X'].shape[3], trks['X'].shape[4])))
    backbone_levels, pyramid_levels, anchor_params = optimal_params
    
    for backbone in backbones:
        if backbone == 'featurenet':
                use_imagenet=False
        else:
            use_imagenet=True
                
        for fpb in fpbs:
            for temporal_mode in temporal_modes:
                model = model_zoo.RetinaMask(backbone=backbone,
                                        use_imagenet=use_imagenet,
                                        panoptic=False,
                                        frames_per_batch=fpb,
                                        temporal_mode=temporal_mode,
                                        num_classes=num_classes,
                                        input_shape=tuple(trks['X'].shape[2:]),
                                        anchor_params=anchor_params,
                                        class_specific_filter=False,
                                        backbone_levels=backbone_levels,
                                        pyramid_levels=pyramid_levels,
                                        norm_method=norm_method)
                prediction_model = model

                model_dir = '/data/models/'
                model_name = backbone + '_' + 'fpb' + str(fpb) + '_' + str(temporal_mode) + '_' + '3T3_HeLa_HEK_RAW_cropped.npz'
                prediction_model.load_weights(model_dir + model_name + '.h5')

                Lstats_allmovies = []

                # Go through each batch (movie) in each dataset
                for batch_num, movie in enumerate(trks['X']):
                    print("batch_num: ", batch_num)
                    Lstats = []

                    # Predict on the raw data
                    X_test_temp = np.expand_dims(movie, axis=0)
                    y_test_temp = np.expand_dims(trks['y'][batch_num], axis=0)
                    print("X_test_temp.shape", X_test_temp.shape)
                    test_images = run_watershed_model.predict(X_test_temp) #[-1]
                    test_images_fgbg = run_fgbg_model.predict(X_test_temp) #[-1]
                    # 
                    # Postprocessing
                    # Collapse predictions into semantic segmentation mask
                    argmax_images = []
                    for frame_num, frame in enumerate(test_images):
                        max_image = np.argmax(frame, axis=-1)
                        argmax_images.append(max_image)
                    argmax_images = np.array(argmax_images)
                    argmax_images = np.expand_dims(argmax_images, axis=-1)

                    # threshold the foreground/background
                    # and remove background from watershed transform
                    fg_thresh = test_images_fgbg[..., 1] > threshold
                    fg_thresh = np.expand_dims(fg_thresh.astype('int16'), axis=-1)
                    argmax_images_post_fgbg = argmax_images * fg_thresh
                    watershed_images = []

                    n_movies = argmax_images_post_fgbg.shape[0]
                    n_frames = argmax_images_post_fgbg.shape[1]

                    for i in range(n_movies):
                        watershed_frames = []
                        for j in range(n_frames):
                            image = fg_thresh[i,j,...,0]
                            semantic = fg_thresh[i,j,...]
                            distance = argmax_images_post_fgbg[i,j,...,0]

                            markers = label(distance == 3)
                            labels = watershed(-distance, markers, mask=image)
                            watershed_frames.append(labels)
                        watershed_images.append(np.stack(watershed_frames, axis=0))

                    watershed_images = np.stack(watershed_images, axis=0)
                    watershed_images = np.expand_dims(watershed_images, axis=-1)
                    print("watershed_images.shape", watershed_images.shape)
                    print("y_test_temp.shape", y_test_temp.shape)

                    # Remove small objects from GT for comparison
                    small_objects_threshold=100
                    for i in range(watershed_images.shape[0]):
                        for j in range(watershed_images.shape[1]):
                            # Remove small objects from GT for comparison
                            GT_image = y_test_temp[i, j, :, :, :]
                            watershed_image = watershed_images[i, j, :, :, :]
                            GT_image = np.expand_dims(GT_image, axis=0)
                            watershed_image = np.expand_dims(watershed_image, axis=0)
                            GT_image = morphology.remove_small_objects(GT_image.astype('uint16'), min_size=small_objects_threshold)

                            pp_watershed_image = morphology.remove_small_objects(
                                                    watershed_image.astype('uint16'), 
                                                    min_size=small_objects_threshold)

                            # Accuracy from metrics package
                            filename = CELL_TYPE_NAME[set_num] + BASE_NAME + '{}'.format(i)

                            m = metrics.Metrics(model_name = filename)
                            m.calc_object_stats(GT_image, pp_watershed_image)

                            Lstats.append(m.stats)

                    Lstats_allmovies.append(Lstats)

                Lstats_AllDatasets.append(Lstats_allmovies)
            save_file_name = backbone + '_' + 'fpb' + str(fpb) + '_' + str(temporal_mode) + '_'
            df_3T3    = pd.concat([pd.concat(Lstats_AllDatasets[0][i]) for i in range(len(Lstats_AllDatasets[0]))])
            df_3T3.to_csv(save_file_name + '3T3', sep='\t', encoding='utf-8')
            df_HEK293 = pd.concat([pd.concat(Lstats_AllDatasets[1][i]) for i in range(len(Lstats_AllDatasets[1]))])
            df_HEK293.to_csv(save_file_name + 'HEK293', sep='\t', encoding='utf-8')
            df_HeLa   = pd.concat([pd.concat(Lstats_AllDatasets[2][i]) for i in range(len(Lstats_AllDatasets[2]))])
            df_HeLa.to_csv(save_file_name + 'HeLa', sep='\t', encoding='utf-8')
            df_RAW264 = pd.concat([pd.concat(Lstats_AllDatasets[3][i]) for i in range(len(Lstats_AllDatasets[3]))])
            df_RAW264.to_csv(save_file_name + 'RAW264', sep='\t', encoding='utf-8')
        
                        